# 5: Bioengineering
## This notebook shows the code and calculations used to determine:
### - Production versus inoculation density and photon flux (Fig. 6C and D)
#### The production simulations can use a lot of memory and take a long time. 
#### Running as a separate notebook still requires more than 16GB of system RAM.
#### Options for reducing memory usage (other than being better at coding...):
#### Separate the simulations into sets:
####     1) Break up bof_frac (e.g. run just 0.2 and then 0.6)
####     - Save the output variable to csv (T5_DF_HL.to_csv('HL_data_20percBOF.csv') for the 20% redirect
####     - Restart the kernel and run the 60% redirect and save as csv  (T5_DF_HL.to_csv('HL_data_60percBOF.csv')
####     - Restart the kernel and load in the output data as a dataFrame
####     2) Same as #1 but break up the metabolites into 3 separate csv files
####     - Save the output variable to csv (T5_DF_HL.to_csv('HL_data_chor_h.csv') for the Shikimate pathway overexpression.
####     - Restart the kernel and run the other metabolites and save as csv
####     - Restart the kernel and load in the output data as a dataFrame

In [ ]:
import pandas as pd
import numpy as np
import csv
import cobra
import math as m
from cobra.core.metabolite import elements_and_molecular_weights
elements_and_molecular_weights['R']=0.0
elements_and_molecular_weights['Z']=0.0
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()
import fcns.modeling_functions as fcns

### Fig. 6 C and D

In [ ]:
## HL samples, Fig. 6C
mets = {'chor_h':224.17, 'ipdp_h':243.07,'hdca_h':255.42}
bof_frac = [0.2,0.6]

base_model = cobra.io.load_json_model('HL_base_iLB1035.json')
(yII_biomass_HL, yII_model_HL) = fcns.simulate(base_model,'HL',photon_const=True,Po_const=True,
                                         YII_const=True,D1_const=True,DM20=False,ngam_comp='mito')

time_interval = 360 ## Note, sims for figure 6 used a 60 min interval,
                    ## but for demonstration we use 360 here
interval = [str(a) for a in range(time_interval,7200+time_interval,time_interval)]  

## Note, this inoc array is a little more sparse than the figure in the paper to save computation
### time and resources.
inoc_array = [100000.00,400000.00,800000.00,1400000.00,2800000.00,5000000.00,10000000.00,
              20000000.00,30000000.00,40000000.00,50000000.00,60000000.00,70000000.00,
              80000000.00,100000000.00,120000000.00,140000000.00,160000000.00,180000000.00,
              195000000.00]

T5_DF_HL = pd.DataFrame(index=inoc_array)

for metab,metgfw in mets.items():
    met_eng = metab
    model = yII_model_HL.copy()
    light = 'HL'
    model = fcns.target_met(model,met_eng)
    
    for bfrac in bof_frac:
        production_DF = pd.DataFrame(columns=interval)
        for inn in inoc_array:
            temp_DF = fcns.get_prod_env(model,met_eng,bfrac,inn,light, time_interval)
            temp_DF = temp_DF.drop(['0'],axis= 1)
            production_DF.loc[inn]=temp_DF.values[0] 

        col_id = met_eng+'_'+light+'_'+str(bfrac)
        T5_DF_HL[col_id]=(production_DF['7200'].values)*metgfw/1000./(bfrac*100)
    
    


In [ ]:
T5_DF_HL.to_csv('HL_T=5days_mgProd_v_inoc.csv')

## At this point, if you are low on RAM, restart the kernel and then execute the code below.  Otherwise, proceed with the Low Light simulations below.

In [ ]:
import pandas as pd
import numpy as np
import csv
import cobra
import math as m
from cobra.core.metabolite import elements_and_molecular_weights
elements_and_molecular_weights['R']=0.0
elements_and_molecular_weights['Z']=0.0
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()
import fcns.modeling_functions as fcns

In [ ]:
## LL samples, Fig. 6D
mets = {'chor_h':224.17, 'ipdp_h':243.07, 'hdca_h':255.42}
bof_frac = [0.2,0.6]

base_model = cobra.io.load_json_model('LL_base_iLB1035.json')
(yII_biomass_LL, yII_model_LL) = fcns.simulate(base_model,'LL',photon_const=True,Po_const=True,
                                         YII_const=True,D1_const=True,DM20=False,ngam_comp='mito')

time_interval = 360 ## Note, sims for figure 6 used a 60 min interval,
                    ## but for demonstration we use 360 here
interval = [str(a) for a in range(time_interval,7200+time_interval,time_interval)]    

## Note, this inoc array is a little more sparse than the figure in the paper to save computation
### time and resources.
inoc_array = [100000., 500000., 800000., 1500000., 3000000., 6000000., 8500000., 
              10000000., 12000000., 15000000., 17500000., 20000000., 25000000.,
              30000000., 35000000., 40000000., 50000000., 60000000.]

T5_DF_LL = pd.DataFrame(index=inoc_array)

for metab,metgfw in mets.items():
    met_eng = metab
    model = yII_model_LL.copy()
    light = 'LL'
    model = fcns.target_met(model,met_eng)
    
    for bfrac in bof_frac:
        production_DF = pd.DataFrame(columns=interval)
        for inn in inoc_array:
            temp_DF = fcns.get_prod_env(model,met_eng,bfrac,inn,light, time_interval)
            temp_DF = temp_DF.drop(['0'],axis= 1)
            production_DF.loc[inn]=temp_DF.values[0] 

        col_id = met_eng+'_'+light+'_'+str(bfrac)
        T5_DF_LL[col_id]=(production_DF['7200'].values)*metgfw/1000./(bfrac*100)
    

In [ ]:
T5_DF_LL.to_csv('LL_T=5days_mgProd_v_inoc.csv')

In [ ]:
T5_DF_HL = pd.read_csv('HL_T=5days_mgProd_v_inoc.csv',index_col=0)
x = T5_DF_HL.index.values
plt.figure(figsize=(5,5))
for c in T5_DF_HL.columns:
    y = T5_DF_HL[c].values    
    plt.scatter(x=x,y=y)


In [ ]:
T5_DF_LL = pd.read_csv('LL_T=5days_mgProd_v_inoc.csv',index_col=0)
x = T5_DF_LL.index.values
plt.figure(figsize=(5,5))
for c in T5_DF_LL.columns:
    y = T5_DF_LL[c].values
    plt.scatter(x=x,y=y)